In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt


2024-03-12 21:50:25.598838: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-12 21:50:25.598854: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:


tfk = tf.keras
tfkl = tf.keras.layers



In [3]:
def build_model(hp, ):
        
    vectorized_texts = tfkl.TextVectorization(
        max_tokens=vocab_size, 
        standardize="lower_and_strip_punctuation",
        split="whitespace", ngrams=1, 
        vocabulary = vocab_data,
        output_mode="int", ragged=True,
    )
    
#     vectorized_texts.adapt(data=text_dataset, batch_size=8, steps=None)

    embedings = tfkl.Embedding(
        input_dim=vocab_size, 
        output_dim=hp.Int('units', min_value=10, max_value=100, step=10)
    )(vectorized_texts)
    
    encoded = tfkl.LSTM(
        units=hp.Int('units', min_value=2, max_value=100, step=5), 
        activation=hp.Choice("activation", ["relu", "tanh"]), 
        dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
        name="encoder1"
    )(embedings)
    
    encoded = tfkl.LSTM(
        units=hp.Int('units', min_value=2, max_value=100, step=5), 
        activation=hp.Choice("activation", ["relu", "tanh"]), 
        dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
        name="encoder2"
    )(encoded)
    
    decoded = tfkl.LSTM(
        units=hp.Int('units', min_value=2, max_value=100, step=5), 
        activation=hp.Choice("activation", ["relu", "tanh"]), 
        dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
        name="decoder1"
    )(encoded)
    
    decoded = tfkl.LSTM(
        units=hp.Int('units', min_value=2, max_value=100, step=5), 
        activation=hp.Choice("activation", ["relu", "tanh"]), 
        dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
        name="decoder2"
    )(decoded)
    
    mdl = tfk.models.Model(vectorized_texts, decoded2)
    opt = tfk.optimizers.SGD(
        learning_rate=hp.Float('learning_rate', min_value=0.0001, max_value=0.1, step=0.1)
    )
    mdl.compile(optimizer=opt, loss="mse", )
    
    return mdl
    

    

In [4]:
# from tensorflow.python import keras
# import numpy as np

# SEQUENCE_LEN = 45
# LATENT_SIZE = 20
# EMBED_SIZE = 50
# VOCAB_SIZE = 100

# inputs = keras.layers.Input(shape=(SEQUENCE_LEN,), name="input")

# embedding = keras.layers.Embedding(output_dim=EMBED_SIZE, input_dim=VOCAB_SIZE, input_length=SEQUENCE_LEN, trainable=True)(inputs)

# encoded = keras.layers.Bidirectional(keras.layers.LSTM(LATENT_SIZE), merge_mode="sum", name="encoder_lstm")(embedding)
# decoded = keras.layers.RepeatVector(SEQUENCE_LEN, name="repeater")(encoded)
# decoded = keras.layers.Bidirectional(keras.layers.LSTM(EMBED_SIZE, return_sequences=True), merge_mode="sum", name="decoder_lstm")(decoded)
# autoencoder = keras.models.Model(inputs, decoded)
# autoencoder.compile(optimizer="sgd", loss='mse')
# autoencoder.summary()

In [5]:

medium_movies_df = pd.read_csv("../data/medium_movies_data.csv",)
print(medium_movies_df.shape)


(2262, 19)


In [6]:


labels = medium_movies_df.Genre.values
print(labels)
print(labels.shape)



['Musical' 'Action' 'Action' ... 'Western' 'Western' 'Mystery & thriller']
(2262,)


In [7]:

text_dataset = medium_movies_df.Synopsis.values
print(text_dataset)
print(len(text_dataset))



["A story of love and resilience based on the novel and the Broadway musical, THE COLOR PURPLE is a decades-spanning tale of one woman's journey to independence. Celie faces many hardships in her life, but ultimately finds extraordinary strength and hope in the unbreakable bonds of sisterhood."
 'In The Beekeeper, one man\'s brutal campaign for vengeance takes on national stakes after he is revealed to be a former operative of a powerful and clandestine organization known as "Beekeepers".'
 'After failing to defeat Aquaman the first time, Black Manta wields the power of the mythic Black Trident to unleash an ancient and malevolent force. Hoping to end his reign of terror, Aquaman forges an unlikely alliance with his brother, Orm, the former king of Atlantis. Setting aside their differences, they join forces to protect their kingdom and save the world from irreversible destruction.'
 ...
 'Leaving Bolivia and heading back to the U.S., the outlaw formerly known as Butch Cassidy (Sam Shep

In [8]:

vocabs = [np.unique(i.lower().strip().split(" ")).tolist() for i in text_dataset]



In [9]:
vocab = []
for t in vocabs:
    print(t)
    for tt in set(t):
        print(tt)
        if tt not in vocab:
            vocab.append(tt)
        

['a', 'and', 'based', 'bonds', 'broadway', 'but', 'celie', 'color', 'decades-spanning', 'extraordinary', 'faces', 'finds', 'hardships', 'her', 'hope', 'in', 'independence.', 'is', 'journey', 'life,', 'love', 'many', 'musical,', 'novel', 'of', 'on', 'one', 'purple', 'resilience', 'sisterhood.', 'story', 'strength', 'tale', 'the', 'to', 'ultimately', 'unbreakable', "woman's"]
celie
life,
one
ultimately
based
novel
woman's
sisterhood.
broadway
bonds
independence.
a
tale
purple
and
love
many
unbreakable
extraordinary
to
finds
story
the
decades-spanning
strength
hope
in
hardships
but
her
of
journey
is
on
faces
color
musical,
resilience
['"beekeepers".', 'a', 'after', 'and', 'as', 'be', 'beekeeper,', 'brutal', 'campaign', 'clandestine', 'for', 'former', 'he', 'in', 'is', 'known', "man's", 'national', 'of', 'on', 'one', 'operative', 'organization', 'powerful', 'revealed', 'stakes', 'takes', 'the', 'to', 'vengeance']
known
operative
one
former
national
"beekeepers".
organization
campaign
a
pow

passion,
find
true
a
and
life
expected.
soul.
it
have
someone
joe
what
to
the
travels
when
discovers
good.
another
band
but
whose
he
middle-school
--
gone
is
means
way
he's
hasn't
soon
['"big"', '(emmy', '(harvey', '(oscar®', '(wagner', 'a', 'academy', 'adventure', 'ahead', 'aided--against', 'along', 'an', 'and', 'animation', 'antonio', 'as', 'ask', 'award®', 'back', 'bad', 'banderas', 'be', 'bears', 'better', 'big', 'black', 'boots', 'bounty', 'burned', 'but', 'captivating', 'chatty,', 'cheerful', 'count', 'crime', 'daring', 'decade,', 'discovers', 'disregard', 'do', 'dreamworks', 'eight', 'embarks', 'epic', "everyone's", 'fall,', 'family,', 'favorite', 'fear-defying', 'feline', 'find', 'first', 'florence', 'for', 'forest', 'former', 'from', 'getting', 'goldilocks', 'grandest', 'guillén,', 'has', 'have', 'hayek).', 'he', 'help', 'heroes', 'himself', 'his', 'horner', 'humble', 'hunter,', 'in', 'into', 'jack', 'john', 'journey', 'judgment--by', 'kitty', 'leche-loving,', 'left,', 'life',

and
family.
downriver,
ochoa)
shines
dino
battered
arlo
ends
with
boy
to
away
epic
wiped
miles
unlikely
the
when
fortune
together,
65
(jeffrey
arlo,
bruised,
(jack
friendship.
he
home.
ago.
spot
two
poor
out
reunite
on
for
mcdormand)
million
wright,
years
rainstorm
an
frightened
from
['--', 'a', 'again.', 'and', 'as', 'care', 'challenges', 'children.', 'computer', 'crime', 'cybercriminal', 'dash', 'dastardly', 'deavor', 'elastigirl', 'enlists', 'ever', 'fall', 'fight', 'greatest', 'guru', 'headaches,', 'him', 'his', 'home', 'hypnotizing', 'in', 'incredible', 'jack-jack', 'launches', 'leaves', 'love', 'make', 'mr.', 'named', 'new', 'of', 'offer', 'once', 'one', 'plan', 'public', 'rambunctious', 'screens.', 'screenslaver', 'set', 'staying', 'superheroes', 'taking', 'telecommunications', 'that', 'the', 'three', 'through', 'to', 'violet,', 'winston', 'with', 'world']
telecommunications
guru
that
his
with
deavor
one
launches
care
once
staying
superheroes
named
plan
a
taking
leaves
and
scree

depression,
evolution.
revolutionary
strongly
it
leaving
finish
by
sinks
to
associates
darwin
galvanized
about
him
the
while
god,
himself
deep
devastated
book
work,
in
emma
wife,
charles
of
urge
objects,
annie,
bettany)
darwin's
has
into
bring
an
daughter,
death
['balasaheb', 'life', 'of', 'thackeray.', 'the']
the
of
life
balasaheb
thackeray.
['(jamal', '1997,', 'a', 'about', 'after', 'and', 'artist.', 'as', 'b.i.g.,', 'biggie', 'born', 'brooklyn,', 'career', 'catches', 'charts.', 'christopher', 'death', 'dubbing', 'eventually', 'for', 'gift', 'he', 'him', 'himself', 'his', 'in', 'into', 'is', 'later,', 'leaving', 'life', 'march', 'n.y.,', 'notorious', 'of', 'often', 'parlays', 'party.', 'raised', 'rap', 'raps', 'shot', 'smalls,', 'so', 'storytelling', 'street', 'subject', 'take', 'tales', 'the', 'then', 'to', 'top', 'up', 'violent', 'wallace', "wallace's", 'when', 'with', 'woolard)']
parlays
shot
b.i.g.,
his
with
notorious
up
subject
party.
1997,
woolard)
a
brooklyn,
later,
(jamal
tak

sudeikis),
guidance
at
sensation
united
(stephan
boycott
man
he's
seizes
numerous
from
gains
['an', 'artist', 'canadian', 'develops', 'everett', 'fishmonger', 'folk', 'lewis', 'lewis.', 'maud', 'reclusive', 'romance', 'unlikely', 'with']
lewis.
reclusive
fishmonger
canadian
maud
with
artist
an
everett
develops
folk
lewis
romance
unlikely
["'50s,", '1965', 'a', 'activist', 'and', 'assassination', 'became', 'black', 'bottom', 'controversial', 'during', 'for', 'he', 'his', 'hit', 'imprisonment', 'in', 'islam.', 'leader', 'left', 'legacy', 'liberation.', 'muslim', 'nation', 'of', 'pride.', 'racial', 'self-determination', 'struggle', 'the', 'then', 'to', 'tribute']
his
hit
self-determination
struggle
black
a
imprisonment
islam.
and
nation
pride.
to
1965
the
became
liberation.
muslim
leader
then
during
in
assassination
he
legacy
controversial
of
racial
'50s,
bottom
for
left
activist
tribute
['a', 'accident,', 'after', 'alcohol.', 'and', 'artist', 'budding', 'by', 'callahan', 'car', 'cartoons

...

(richard
up
lawyer,
their
each
nightclub
defend
zellweger)
a
outdone
will
zeta-jones)
and
well
also
as
prison,
winds
husband,
takes
chicago's
to
woman
set
circus
the
when
fight
(renée
media
slickest
against
case
turning
in
neither
but
celebrity.
flynn
--
her
be
velma
of
billy
is
on
for
into
other
sensation
her.
headlines.
public
(catherine
['(emily', '(james', '(meryl', 'a', 'and', 'are', 'as', 'baker', 'beauty:', 'before', 'blood-red', 'blue', 'blunt)', 'cape,', 'childless.', 'cinderella,', 'corden)', 'corn,', 'cow,', 'curse', 'days', 'during', 'each', 'find', 'forest', 'fulfill', 'gold.', 'hair', 'his', 'hood,', 'ingredients', 'into', 'jack,', 'journey,', 'little', 'meet', 'milk-white', 'moon,', 'of', 'on', 'once-beautiful', 'one', 'quest', 'rapunzel', 'red', 'restore', 'result', 'reverse', 'riding', 'rise', 'slipper', 'spell', 'streep),', 'that', 'the', 'their', 'they', 'three', 'to', 'venture', 'wife', 'will', 'wish.', 'witch', "witch's", 'yellow']
are
once-beautiful
streep),
milk-

connection
safe
embrace
does
fibrosis
distance
hospital
intensifies,
though
a
attraction.
all
meets
routines,
and
charming
cystic
full
life
as
them.
teen
to
the
dictate
seventeen-year-old
must
she
when
restrictions
window
put
instant
illness.
spends
between
in
rules
there's
--
her
maintain
throw
out
of
temptation
is
get
has
they
self-control
an
flirtation,
same
so
impossibly
['(channing', '(rachel', '(sam', '(scott', 'a', 'accident', 'also', 'and', 'are', 'awakes,', 'bond', 'but', 'car', 'coma.', 'confusing', 'despite', 'determined', 'difficult', "doesn't", 'feelings', 'fiance', 'for', 'former', 'happily', 'harbors', 'has', 'her', 'idyllic', 'in', 'is', 'jessica', 'know', 'lange)', 'leaves', 'leo', 'leo.', 'life', 'loss', 'marriage.', 'married', 'mcadams)', 'memory', 'neill,', 'newlyweds,', 'obstacles,', 'paige', 'parents', 'rebuild', 'relationship', 'renew', 'severe', 'shatters', 'she', 'speedman).', 'still', 'suddenly', 'tatum)', 'their', 'these', 'to', 'together', 'when', 'with']
ar

kyle
adam
always
along
so
malignant
['(carey', '(matthias', '(michael', '(tom', 'a', 'and', 'bachelor', 'beauty', 'choice', 'dashing', 'different', 'farmer', 'has', 'headstrong', 'her', 'mulligan)', 'of', 'older', 'prosperous,', 'schoenaerts),', 'sheen).', 'sheep', 'soldier', 'sturridge)', 'suitors:', 'three', 'victorian']
bachelor
headstrong
different
a
dashing
(michael
older
and
victorian
sheen).
prosperous,
sturridge)
mulligan)
(tom
beauty
her
(matthias
of
choice
suitors:
(carey
farmer
has
soldier
sheep
schoenaerts),
three
['(owen', '(rachel', '--', 'a', "age's", 'alone.', 'along,', 'and', 'apparently', 'art', 'aspiring', 'back', 'becomes', 'city', 'cultural', 'dissatisfied', 'encounters', 'excursion,', 'familiar', 'fiancee', 'for', 'gil', 'group', 'has', 'he', 'heroes', 'him', 'his', 'icons', 'in', 'is', 'jazz', 'late-night', 'literature.', 'mcadams),', 'more', 'night', 'novelist.', 'of', 'on', 'one', 'paris', 'past,', 'pender', 'present.', 'revelers,', 'screenwriter', 'some', 'spe

save
habitat
an
man
intelligent
massive
from
now,
['(sharlto', '--', '9,', '9.', 'a', 'advanced', 'african', 'agent', 'ago,', 'aid,', 'aliens', "aliens'", 'alter', 'anything', 'are', 'area', 'arrive', 'begins', 'but', 'by', 'called', 'can', 'company', 'conquer', 'contracts', 'copley)', 'district', 'dna,', 'do', 'dying', 'earth', 'field', 'find', 'from', 'give', 'he', 'hide:', 'his', 'humans', 'in', 'is', 'managed', 'master', 'multi-national', 'mysterious', 'not', 'on', 'one', 'only', 'or', 'place', 'planet.', 'refuge', 'separated', 'south', 'technology.', 'that', 'the', 'their', 'there', 'thirty', 'to', 'unconcerned', 'united,', 'virus', 'welfare', 'when', 'which', 'will', 'with', 'years']
are
unconcerned
planet.
which
that
his
with
alter
virus
one
master
only
agent
their
(sharlto
arrive
not
find
managed
technology.
a
refuge
will
field
ago,
aliens
do
area
african
anything
begins
dna,
dying
multi-national
mysterious
thirty
there
united,
by
to
humans
the
9,
when
company
south
contracts
c

two
he
don't
come,
(mario
officers
train
odd,
wars.
holdover.
stitch
of
prepares
view
is
on
for
has
previously
soldier
surrounded
invades
combat
they
undisciplined
an
experience
believe
['(carice', '(sebastian', 'a', 'after', 'assuming', 'becomes', 'but', 'de', 'death,', 'ellis', 'enemy.', 'escaping', 'falling', 'finds', 'gestapo', 'her', 'herself', 'houten)', 'in', 'is', 'jewish', 'koch).', 'love', 'ludwig', 'mission', 'mortal', 'name', 'named', 'narrowly', 'of', 'officer', 'order', 'part', 'rachel', 'resistance,', 'rosenthal', 'seduce', 'soon', 'successful', 'superiors', 'the', 'to', 'van', 'vries.', 'with', 'young']
officer
(carice
young
ellis
escaping
jewish
with
falling
part
herself
de
named
resistance,
a
love
enemy.
order
seduce
van
to
finds
becomes
(sebastian
the
rachel
assuming
death,
after
in
mortal
narrowly
but
koch).
her
superiors
vries.
rosenthal
mission
of
is
successful
gestapo
houten)
name
ludwig
soon
['(tang', '(tony', 'a', 'agent', 'an', 'assassinate', 'assigned', 'beco

In [10]:

# vocab




In [11]:


vocab_size = sum([len(set(text_dataset[i].split())) for i in range(len(text_dataset))]) + 100
print(vocab_size)



125780


In [12]:
vectorized_texts = tfkl.TextVectorization(
    max_tokens=vocab_size, 
    standardize="lower_and_strip_punctuation",
    split="whitespace", ngrams=1, 
    output_mode="int", ragged=True,
    vocabulary=vocab,
)

type(vectorized_texts)


2024-03-12 21:50:36.965207: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-12 21:50:36.965226: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SorooshShalileh): /proc/driver/nvidia/version does not exist
2024-03-12 21:50:36.965376: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


keras.layers.preprocessing.text_vectorization.TextVectorization

In [17]:


inputs = tf.keras.Input(shape=(1,), dtype=tf.string) 
 
vectorized_texts = tfkl.TextVectorization(
    max_tokens=vocab_size, 
    standardize="lower_and_strip_punctuation",
    split="whitespace", ngrams=1, 
    output_mode="int", ragged=True,
    vocabulary=vocab,
)(inputs)

embedings = tfkl.Embedding(
    input_dim=vocab_size, 
    output_dim=40,  # hp.Int('units', min_value=10, max_value=100, step=10)
)(vectorized_texts)

encoded = tfkl.LSTM(
    units=20,  # hp.Int('units', min_value=2, max_value=100, step=5), 
    activation="relu",  # hp.Choice("activation", ["relu", "tanh"]), 
#     dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
    name="encoder1"
)(embedings)

encoded = tfkl.LSTM(
    units=10, # hp.Int('units', min_value=2, max_value=100, step=5), 
    activation="relu",  # hp.Choice("activation", ["relu", "tanh"]), 
    name="encoder2"
)(encoded)

decoded = tfkl.LSTM(
    units=10,  # hp.Int('units', min_value=2, max_value=100, step=5), 
    activation="relu",  #hp.Choice("activation", ["relu", "tanh"]), 
#     dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
    name="decoder1"
)(encoded)

decoded = tfkl.LSTM(
    units=20, # hp.Int('units', min_value=2, max_value=100, step=5), 
    activation="relu", # hp.Choice("activation", ["relu", "tanh"]), 
#     dropout=  # hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
    name="decoder2",
)(decoded)

mdl = tfk.models.Model(vectorized_texts, decoded2)
opt = tfk.optimizers.SGD(
    learning_rate=hp.Float('learning_rate', min_value=0.0001, max_value=0.1, step=0.1)
)
mdl.compile(optimizer=opt, loss="mse", )

return mdl



ValueError: Input 0 of layer "decoder1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 20)

In [ ]:

# Load GloVe embeddings into a dictionary 
def load_embeddings(file_path):
    embeddings = {} 
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f: 
            values = line.split() 
            word = values[0] 
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector 
            
    return embeddings 
        

In [ ]:

glove_embeddings_path = './glove.6B.50d.txt' # Adjust the path to your downloaded GloVe fileglove_embeddings = load_embeddings(glove_embeddings_path)

In [ ]:


glove_embeddings = load_embeddings(glove_embeddings_path)



In [ ]:


# Accessing word embeddings
word = 'example'
if word in glove_embeddings:
    embedding = glove_embeddings[word]
    print(f"Embedding for '{word}': {embedding}")
else:
    print(f"'{word}' not found in embeddings")

# Finding similarity between word embeddings
from scipy.spatial.distance import cosine

word1 = 'king'
word2 = 'queen'
similarity = 1 - cosine(glove_embeddings[word1], glove_embeddings[word2])
print(f"Similarity between '{word1}' and '{word2}': {similarity}")



In [ ]:

# train_ds = tf.data.Dataset.from_tensor_slices((text_dataset, labels))

# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)



In [ ]:

vectorize_layer = tfkl.TextVectorization(
    max_tokens=vocab_size, standardize="lower_and_strip_punctuation",
    split="whitespace", ngrams=1, output_mode="int", ragged=True, 
    
)


In [ ]:



vectorize_layer.adapt(data=text_dataset, batch_size=8, steps=None)




In [ ]:

model = tf.keras.models.Sequential()



In [ ]:

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))



In [ ]:


model.add(vectorize_layer)




In [ ]:

embeding_layer = tfkl.Embedding(input_dim=vocab_size, output_dim=3)
model.add(embeding_layer)



In [ ]:


input_data = [["Soroosh "], ["qux baz"],
              ["A story of love and resilience based on the novel and the Broadway "]
             ]



In [ ]:

preds = model.predict(input_data)
# preds


In [ ]:

for pred in preds:
    print(len(pred))
    print(pred)
    

In [ ]:

for l in vectorize_layer.get_vocabulary():
    print(l)


In [ ]:


vectorize_layer.is_adapted


In [ ]:
len(vectorize_layer.get_vocabulary())

In [ ]:

vectorize_layer.vocabulary_size()




In [ ]:
vectorize_layer.get_weights()


### LSTM-AE



In [ ]:


# code for Glove word embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

x = {'text', 'the', 'leader', 'prime', 'natural', 'language'}

# create the dict.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_dataset)

# number of unique words in dict.
print("Number of unique words in dictionary=", len(tokenizer.word_index))
print("Dictionary is = ", tokenizer.word_index)

# download glove and unzip it in Notebook.
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip





In [ ]:

# vocab: 'the': 1, mapping of words with
# integers in seq. 1,2,3..
# embedding: 1->dense vector
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1 # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab




In [ ]:
# tokenizer.index_word

In [ ]:
# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
    './glove.6B.50d.txt',
    tokenizer.word_index,
    embedding_dim
)

print("Dense vector for first word is => ", embedding_matrix_vocab[10])


